In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests
import csv
import re
from collections import defaultdict

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result']

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def get_size(size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony-open-staking/validators_with_page?active=true&page=0&search=&size={}&sortOrder=desc&sortProperty=expectedReturns".format(size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['total']

In [6]:
def get_validator(page, size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony-open-staking/validators_with_page?active=false&page={}&search=&size={}&sortOrder=desc&sortProperty=expectedReturns".format(page, size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['validators']

In [18]:
def read_csv(csv_file) -> (dict, list):
    r = requests.get(csv_file)
    s = [x.decode(encoding) for x in r.content.splitlines()]
    d = defaultdict(list)
    v = []
    dup_list = []
    for line in csv.reader(s):
        group = line[1].strip()
        email = line[3].strip()
        address = line[7].strip()
        if group in groups and re.match('one1', address) != None:
            d[group].append(address)
            v.append(address)
    return d, v

In [8]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            if v - map1[k] != 0:
                map3[k] = v - map1[k]
    return map3

In [9]:
def getNewValidatorInfo(address, df):
    index = df[df['address'] == address].index[0]
    return df.loc[index]['name'], df.loc[index]['security-contact'], df.loc[index]['website']

In [10]:
bls_key = []
availability = []
name = []
epos_status = []
apr = []
delegation = []
committee = []
website = []
details = []
address = []
security = []
identity = []
dels = defaultdict(int)
# get the accumualted reward in current block
block = getBlockNumber()
next_block = block + 1
validator_infos = getAllValidatorInformation()
for info in validator_infos:  
    for d in info['validator']['delegations']:
        del_address = d['delegator-address']
        reward = d['reward']/1e18
        dels[del_address] += reward
# get the new accumulated reward in next block
while block < next_block:
    block = getBlockNumber()
validator_infos = getAllValidatorInformation()
new_dels = defaultdict(int)
for info in validator_infos:
    if not info['metrics']:
        bls_key.append(np.nan)
    else:
        bls_key.append(len(info['metrics']['by-bls-key']))
    if not info['current-epoch-performance']:
        availability.append(np.nan)
    else:
        availability.append(info['current-epoch-performance']['current-epoch-signing-percent'])
    name.append(info['validator']['name'])
    epos_status.append(info['epos-status'])
    apr.append(info['lifetime']['apr'])
    delegation.append(len(info['validator']['delegations'][0]))
    committee.append(info['currently-in-committee'])
    website.append(info['validator']['website'])
    details.append(info['validator']['details'])
    address.append(info['validator']['address'])
    security.append(info['validator']['security-contact'])
    identity.append(info['validator']['identity'])    
    for d in info['validator']['delegations']:
        del_address = d['delegator-address']
        reward = d['reward']/1e18
        new_dels[del_address] += reward
# get the reward the validator earned in that block
reward_dict = diffAndFilter(dels, new_dels)

In [11]:
# get most infos except reward and uptime
df = pd.DataFrame(list(zip( name, address, website, details, security, identity, epos_status, committee, apr, delegation, bls_key,  availability)), \
                  columns =['name','address', 'website','details','security-contact','identity','epos-status', 'currently-in-committee', 'apr', 'delegator-num', 'bls-key-num', 'availability'])
df = pd.concat([df.drop(['availability'], axis=1), df['availability'].apply(pd.Series)], axis=1)
df.drop([0], axis = 1, inplace = True)
df.reset_index(inplace = True, drop = True)
# get reward per block
reward = pd.DataFrame(reward_dict.items(), columns=['address', 'reward'])
df = df.join(reward.set_index("address"), on = 'address')
# get accumulated reward
acc_reward = pd.DataFrame(new_dels.items(), columns=['address', 'acc_reward'])
df = df.join(acc_reward.set_index("address"), on = 'address')
# get uptime
size = get_size(1)
pages = size // 100 + 1
# get the validator info
validator = []
for i in range(pages):
    val = get_validator(i,100)
    validator.extend(val)
uptime = []
address = []
for i in validator:
    uptime.append(i['uptime_percentage'])
    address.append(i['address'])
uptime_df = pd.DataFrame(list(zip(uptime, address)), columns = ['uptime-percentage','address'])
df = df.join(uptime_df.set_index("address"), on = 'address')
df.to_csv('validator_info.csv', index = False)
print("-- Epos Status Summary --")
count = df.groupby('epos-status')['epos-status'].count().reset_index(name = 'count')
elected, eligible, ineligible = count['count'][0], count['count'][1], count['count'][2]
print("Currently elected: %s \nEligible to be elected next epoch: %s \nNot eligible to be elected next epoch: %s \n" % (elected, eligible, ineligible))

-- Epos Status Summary --
Currently elected: 160 
Eligible to be elected next epoch: 2 
Not eligible to be elected next epoch: 198 



In [19]:
csv_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTUUOCAuSgP8TcA1xWY5AbxaMO7OSowYgdvaHpeMQudAZkHkJrf2sGE6TZ0hIbcy20qpZHmlC8HhCw1/pub?gid=0&single=true&output=csv'
encoding = 'utf-8'
groups = ['team', 'p-ops', 'foundational nodes', 'p-volunteer', 'hackers', 'community', 'partners']
by_group, csv_validators = read_csv(csv_link)
new_validators = [x for x in df['address'] if x not in csv_validators]

In [20]:
print("-- New Validators --")
print("New Validators: %d" % len(new_validators))
for i in new_validators:
    name, security, website = getNewValidatorInfo(i, df)
    print("Address: %s, Validator Name: %s, Security Contact: %s, Website: %s" %(i, name, security, website))


-- New Validators --
New Validators: 70
Address: one1y5044jpnt2egetcpuaw4ng3d2egfp92j3cx5rj, Validator Name: Wang Lee Validator 1, Security Contact: info@ankr.com, Website: www.ankr.com
Address: one15mx7cgu93a2pf8lcmatrcn65t253s32rx5l7l0, Validator Name: Bubz, Security Contact: Bubz Hacker Legion, Website: https://i.imgur.com/G6Mruca.jpg
Address: one1cq9hvst9zgc3vxp4gn08yhrhuxxc7q2x5gv6ep, Validator Name: Bubz, Security Contact: Bubz Hacker Legion, Website: https://i.imgur.com/G6Mruca.jpg
Address: one1n0ayywreah5ehwr6lwq4s5582s6xweau2j4wuu, Validator Name: Bubz, Security Contact: Bubz Hacker Legion, Website: https://i.imgur.com/G6Mruca.jpg
Address: one1nhjygh8vdtfcnf7mrapxlehq3uthxm9qcpnfhr, Validator Name: U1_Validator, Security Contact: Bubz Hacker Legion, Website: https://i.imgur.com/G6Mruca.jpg
Address: one196d32mvxkqz5dk4e8a04jwatsf8kmjfgetqf8a, Validator Name: Bubz, Security Contact: Bubz Hacker Legion, Website: https://i.imgur.com/G6Mruca.jpg
Address: one156k98ujrrtxz5r72x753cdx